## Constructing a Minimum-Variance Portfolio

First, we need to import all of the libraries that we'll be using:

* NumPy for linear algebra
* pandas for dataframes and variance/covariance calculations
* yfinance for financial data
* scipy.optimize for solving for the optimal stock weights

In [75]:
# Imports
import numpy as np
import pandas as pd
import yfinance as yf
from scipy import optimize

### Collecting and Preparing the Data

In this example, we'll use the top 10 largest US stocks by market capitalization to build our portfolio. This can easily be changed by adding/removing entries from the tickers list. The other parameters that can be changed and tuned are:

* period: how far back we will go when calculating returns, variances, and covariances.
* interval: the frequency that returns will be calculated over (i.e. daily returns, monthly returns, etc.).
* allow_short: whether or not short selling is allowed, we'll assume that it is free when allowed.

The following code will download a dataframe with the closing price for each of the selected stocks with the given parameters. Then, it will print out the first five rows to verify that the correct data has been downloaded.

In [76]:
### USER PARAMETERS ###

tickers = [
    'AAPL',
    'MSFT',
    'GOOG',
    'AMZN',
    'BRK-B',
    'V',
    'XOM',
    'UNH',
    'JNJ',
    'NVDA',
]

period = '5y' # Options are 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, and max

interval = '1d' # Options are 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, and 3mo

allow_short = True

expected_return = 15 # As a percentage (for example, enter '10' for 10%)

#######################

# Create a list of tuples for each given stock with the format ('AAPL', 'Close') so that we can request only the closing prices
fields = []
for i in range(len(tickers)):
    fields.append((tickers[i], 'Close'))

# Download the data with the user parameters
df = yf.download(
    tickers = tickers,
    period = period,
    interval = interval,
    group_by = 'ticker'
)[fields]

# Print the first five rows
print(df.head())

[*********************100%***********************]  10 of 10 completed
                 AAPL       MSFT       GOOG       AMZN       BRK-B  \
                Close      Close      Close      Close       Close   
Date                                                                 
2018-01-29  41.990002  93.919998  58.778999  70.884003  215.460007   
2018-01-30  41.742500  92.739998  58.184502  71.890999  214.410004   
2018-01-31  41.857498  95.010002  58.497002  72.544502  214.380005   
2018-02-01  41.945000  94.260002  58.384998  69.500000  217.250000   
2018-02-02  40.125000  91.779999  55.595001  71.497498  209.119995   

                     V        XOM         UNH         JNJ       NVDA  
                 Close      Close       Close       Close      Close  
Date                                                                  
2018-01-29  124.839996  88.010002  247.410004  143.679993  61.712502  
2018-01-30  123.550003  86.779999  236.649994  142.429993  60.680000  
2018-01-31  1

Since we're interested in the stocks' returns, not their closing prices, we will create a new dataframe that instead has the percent change for each row (the last row will be dropped since we can't calculate its percent change).

In [77]:
# Find the returns for each row (except the last one) and store it in a new dataframe
returns = df.pct_change().dropna() * 100

# Print the first five rows to verify
print(returns.head())

                AAPL      MSFT      GOOG      AMZN     BRK-B         V  \
               Close     Close     Close     Close     Close     Close   
Date                                                                     
2018-01-30 -0.589429 -1.256389 -1.011412  1.420625 -0.487331 -1.033317   
2018-01-31  0.275493  2.447708  0.537085  0.909020 -0.013991  0.550385   
2018-02-01  0.209046 -0.789391 -0.191468 -4.196737  1.338742  1.199386   
2018-02-02 -4.339015 -2.631024 -4.778620  2.874097 -3.742235 -3.825960   
2018-02-05 -2.498439 -4.118543 -5.045418 -2.793801 -5.891351 -3.837571   

                 XOM       UNH       JNJ      NVDA  
               Close     Close     Close     Close  
Date                                                
2018-01-30 -1.397572 -4.349060 -0.869989 -1.673083  
2018-01-31  0.599221  0.054936 -2.976894  1.268953  
2018-02-01  2.027487 -0.658838  1.324265 -2.156226  
2018-02-02 -5.097116 -1.419946 -1.671198 -2.902285  
2018-02-05 -5.690285 -5.114715 -5.29

### Covariance Matrix

Before we can calculate and optimize the portfolio variance, we need the covariance matrix of the portfolio. The following code will calculate it and store it in a NumPy array for future linear algebra.

In [78]:
# Create covariance matrix and covert from pandas dataframe to NumPy array
covariance = returns.cov().to_numpy()

yearly_returns = returns.mean() * 252

def portfolio_returns(w):
    return (np.sum(w * yearly_returns))

# Print to verify
print(yearly_returns)

AAPL   Close    30.606543
MSFT   Close    24.348068
GOOG   Close    15.774502
AMZN   Close    13.823856
BRK-B  Close     9.938382
V      Close    16.703721
XOM    Close    11.277227
UNH    Close    17.986480
JNJ    Close     5.341281
NVDA   Close    37.615032
dtype: float64


### Optimization

Now it is time to optimize our portfolio using this formulation:

$$minimizeVariance = w \Sigma w^T,$$
$$s.t. \sum_i w_i = 1$$

We also will bound the weights to [0, 1] when shorting is not allowed.

In [79]:
# Objective function
def portfolioVar(w, cov):
    w = np.matrix(w) # Convert w and cov into matrices for easy linear algebra
    cov = np.matrix(cov)
    result = w * cov * w.T # Calculate the objective function
    return result

# Constraint
constraints = (
    {'type': 'eq', 'fun': lambda x:  np.sum(x) - 1},
    {'type': 'eq', 'fun': lambda x: portfolio_returns(x) - expected_return}
    ) # The sum of the array weights is equal to 1

# Bounds
bounds = [()]
if allow_short:
    bounds = tuple((-np.inf, np.inf) for i in tickers)
else:
    bounds = tuple((0, 1) for i in tickers)

Now that we have our objective function, constraint, and bounds, the last thing we need is an initial guess. Since it doesn't need to be a good guess, just a feasible one, we will just use equal weights for all of the stocks.

The following code creates that initial guess, then solves for the optimal stock weights. Finally, it prints out the weights.

In [80]:

# Initial guess, equal weights
w0 = [1 / len(tickers)] * len(tickers)

result = optimize.minimize(
    fun = portfolioVar, # Objective function
    x0 = w0, # Initial guess
    args = covariance, # Covariance matrix for objective function parameters
    method = 'SLSQP', # Sequential least squares programming method
    bounds = bounds, # Bounds
    constraints = constraints # Constraint
)

# Express the optimal results in percentages
result.x = result.x * 100

# Print the optimal weights of each stock
for i in range(len(tickers)):
    print(tickers[i], '=', round(result.x[i], 3), '%')

print('Expected Portfolio Returns = ', portfolio_returns(result.x / 100))

AAPL = 23.208 %
MSFT = 12.796 %
GOOG = -6.166 %
AMZN = 1.294 %
BRK-B = 12.325 %
V = -0.018 %
XOM = 8.015 %
UNH = 10.867 %
JNJ = 39.284 %
NVDA = -1.605 %
Expected Portfolio Returns =  14.999999999887576
